In [1]:
import sys
sys.path.append('./package/')

import networkx as nx

from package.utils import read_items, getItemsPrice
from package.topic import TopicModel
from package.itemset import ItemRelation, ItemsetFlyweight
from package.model import DiffusionModel
from package.algorithm import Algorithm

ITEMS_FILE = r"D:\\論文實驗\\data\\amazon\\sample_items.csv"
items = read_items(ITEMS_FILE)
for key, attr in items.items():
    if "price" not in attr:
        del items[key]
        
topicModel = TopicModel(3)
topicModel.randomTopic(items_id=items.keys())

graph = nx.diamond_graph()

relation = ItemRelation()
relation.construct(ITEMS_FILE)
itemset = ItemsetFlyweight(getItemsPrice(ITEMS_FILE), topicModel, relation)

model = DiffusionModel("amazon in dblp", graph, itemset, threshold=10**(-5))
algo = Algorithm(model, 10)
candidatedCoupons = algo.genAllCoupons(20.0)
print(len(candidatedCoupons))

16


In [2]:
NUM = 16
for i in range(NUM):
    print(candidatedCoupons[i])

12.31, B000VYIUJS, 5.0, B000VYIUJS
12.31, B000VYIUJS, 5.0, B000JX5JGI
12.31, B000VYIUJS, 5.0, B000JX5JGI B000VYIUJS
12.31, B000VYIUJS, 25.0, B000JX5JGI B000VYIUJS
18.99, B000JX5JGI, 5.0, B000VYIUJS
18.99, B000JX5JGI, 5.0, B000JX5JGI
18.99, B000JX5JGI, 5.0, B000JX5JGI B000VYIUJS
18.99, B000JX5JGI, 25.0, B000JX5JGI B000VYIUJS
12.31, B000JX5JGI B000VYIUJS, 5.0, B000VYIUJS
12.31, B000JX5JGI B000VYIUJS, 5.0, B000JX5JGI
12.31, B000JX5JGI B000VYIUJS, 5.0, B000JX5JGI B000VYIUJS
12.31, B000JX5JGI B000VYIUJS, 25.0, B000JX5JGI B000VYIUJS
32.31, B000JX5JGI B000VYIUJS, 5.0, B000VYIUJS
32.31, B000JX5JGI B000VYIUJS, 5.0, B000JX5JGI
32.31, B000JX5JGI B000VYIUJS, 5.0, B000JX5JGI B000VYIUJS
32.31, B000JX5JGI B000VYIUJS, 25.0, B000JX5JGI B000VYIUJS
